In [1]:
import bw2data, bw2io, bw2calc
from premise import *
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [2]:
from premise import __version__
__version__


(2, 2, 6)

In [3]:
# ndb = NewDatabase(scenarios = [
#         {"model":"remind", "pathway":"SSP2-NPi", "year":2005},
#         {"model":"remind", "pathway":"SSP2-NPi", "year":2010},
#         {"model":"remind", "pathway":"SSP2-NPi", "year":2020},
#         {"model":"remind", "pathway":"SSP2-NPi", "year":2030},
#         {"model":"remind", "pathway":"SSP2-NPi", "year":2040},
#         {"model":"remind", "pathway":"SSP2-NPi", "year":2050},
#         {"model":"remind", "pathway":"SSP2-PkBudg1150", "year":2030},
#         {"model":"remind", "pathway":"SSP2-PkBudg1150", "year":2040},
#         {"model":"remind", "pathway":"SSP2-PkBudg1150", "year":2050},
#         {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2030},
#         {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2040},
#         {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
#     ],
#   source_db = 'ecoinvent-3.10.1-cutoff',
#   source_version = "3.10",
#   biosphere_name="biosphere3",
#   key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo='
#  )
#
# ndb.update()
# ndb.write_db_to_brightway()

In [4]:
own_acts = [
    [a["name"], a["reference product"], a["unit"], a["location"]]
    for a in bw2data.Database("wind turbines")
]

ei_acts = [
    [
        "market for concrete, normal strength",
        "concrete, normal strength",
        "cubic meter",	"CH",
    ],
    [
        "market group for electricity, medium voltage",
        "electricity, medium voltage",
        "kilowatt hour",    "RER",
    ],
    [
        "market for reinforcing steel", "reinforcing steel", "kilogram", "GLO"
    ],
    [
        "chromium steel removed by milling, small parts",
        "chromium steel milling, small parts",
        "kilogram",
        "RER",
    ],
    [
        "market for steel, chromium steel 18/8",
        "steel, chromium steel 18/8",
        "kilogram",
        "GLO",
    ],
    [
        "market for transport, freight, lorry, unspecified",
        "transport, freight, lorry, unspecified",
        "ton kilometer",
        "RER",
    ],
    [
        "market for lubricating oil",
        "lubricating oil",
        "kilogram",
        "RoW",
    ]
]

bio_flows = [
    (
        "oxygen", ("air",), "kilogram"
    ),
]

In [5]:
for act in bw2data.Database("wind turbines"):
    for exc in act.exchanges():
        if exc["type"] == "technosphere":
            key = [exc["name"], exc["reference product"], exc["unit"], exc["location"]]
            if key not in ei_acts:
                ei_acts.append(key)


In [6]:
len(own_acts), len(ei_acts)

(24, 49)

In [7]:
import csv
import numpy as np
from pathlib import Path

def create_index(db, filepath):
    filepath = Path(filepath)

    all_acts = []
    for act in own_acts:
        if act not in all_acts:
            all_acts.append((act[0], act[-1], act[2], act[1]))

    for act in ei_acts:
        if act not in all_acts:
            all_acts.append((act[0], act[-1], act[2], act[1]))

    for act in bio_flows:
        if act not in all_acts:
            all_acts.append(act)

    index = {}
    i = 0
    for act in all_acts:
        if act not in index:
            index[act] = i
            i += 1

    with open(filepath / f'dict_inputs_A_matrix.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_NONE, delimiter=';')
        for act in index:
            wr.writerow(act)
    return index

def create_A_matrix(db, filepath, index):

    all_acts = list(index.keys())

    arr = np.zeros((len(all_acts), len(all_acts)))

    for act in bw2data.Database(db):
        key = [act["name"], act["reference product"], act["unit"], act["location"]]
        if key in own_acts:
            ix = index[(key[0], key[-1], key[2], key[1])]
            #print(key[0])
            for exc in act.exchanges():
                if exc["type"] in ["production", "technosphere"]:
                    try:
                        ix_ = index[(exc["name"], exc["location"], exc["unit"], exc["reference product"])]
                    except:
                        print(exc.as_dict())
                        ix_ = 0
                else:
                    try:
                        ix_ = index[(exc["name"], exc["categories"], exc["unit"])]
                    except:
                        print((exc["name"], exc["categories"], exc["unit"]))
                        ix_ = 0

                if exc["type"] == "production":
                    arr[ix_, ix] += exc["amount"]
                else:
                    arr[ix_, ix] += exc["amount"] * -1

    for i in range(arr.shape[0]):
        if arr[i, i] == 0:
            arr[i, i] = 1

    print(f"A matrix shape: {arr.shape}")

    sarr = sparse.csr_matrix(arr)
    sparse.save_npz(Path(filepath) / f'A_matrix.npz', sarr)


In [8]:
def get_midpoint_methods(method_name):

    if method_name == "recipe":
        methods = [
            m for m in bw2data.methods
            if m[0] == 'ReCiPe 2016 v1.03, midpoint (H)'
        ]

        methods.extend(
           [
                ('Cumulative Energy Demand (CED)', 'energy resources: non-renewable','energy content (HHV)'),
                ('Cumulative Energy Demand (CED)', 'energy resources: renewable','energy content (HHV)'),
                ('Cumulative Energy Demand (CED)', 'total', 'energy content (HHV)'),
           ]
        )

        return methods

    if method_name == "ef":
        methods = [
            m for m in bw2data.methods if "ef v3.1" in m[0].lower()
            and "LT" not in m[0]
            and "15804" not in m[0]
        ]
        return methods

def get_endpoint_methods():
    methods = [m for m in bw2data.methods
               if 'recipe 2016' in m[0].lower()
               and 'lt' not in m[0].lower()
               and "(h)" in m[0].lower()
               and "endpoint" in m[0]
              ]
    return methods

In [9]:
import pandas as pd
from pathlib import Path
from scipy import sparse
import bw2calc


LIST_DATABASES = [
    "ei_cutoff_3.10_remind_SSP2-NPi_2005 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-NPi_2010 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-NPi_2020 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-NPi_2030 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-NPi_2040 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-NPi_2050 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-PkBudg1150_2030 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-PkBudg1150_2040 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-PkBudg1150_2050 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-PkBudg500_2030 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-PkBudg500_2040 2025-02-11",
	"ei_cutoff_3.10_remind_SSP2-PkBudg500_2050 2025-02-11",
    "ecoinvent-3.10.1-cutoff"
]

def generate_calculated_values(ei_acts, index, method_name, indicator, filepath):

    if indicator == "midpoint":
        methods = get_midpoint_methods(method_name)
    else:
        methods = get_endpoint_methods()

    print(f"Number of methods: {len(methods)}")

    for l in LIST_DATABASES:
        FU = []
        for act in bw2data.Database(l):
            if[act['name'], act['reference product'], act['unit'], act['location']] in ei_acts:
                FU.append({act:1})

        print(f"Calculating impacts for {len(FU)} activities")
        bw2data.calculation_setups['multiLCA'] = {'inv': FU, 'ia': methods}
        myMultiLCA = bw2calc.MultiLCA('multiLCA')

        results = {
            (f["name"], f["location"], f["unit"], f["reference product"]): r
            for f, r in zip([list(x.keys())[0]
            for x in myMultiLCA.func_units], np.squeeze(myMultiLCA.results).tolist())
        }

        B = np.zeros(
            (
                len(methods) + (1 if method_name=="recipe" else 0), # + noise
                len(index)
            )
        )

        print(f"B shape: {B.shape}")

        for key, value in results.items():
            idx = index[key]
            if method_name=="recipe":
                B[: -1, idx] = value
            else:
                B[:, idx] = value

        for i, method in enumerate(methods):
            m = bw2data.Method(method)

            for code, cf in m.load():
                try:
                    act = bw2data.get_activity(code)
                    key = (act["name"], act["categories"], act["unit"])
                    if key in bio_flows:
                        flow = [f for f in bio_flows if f==key][0]
                        idx = index[flow]
                        B[i, idx] = cf
                except:
                    print(f"Skipping: {code} in {method}")
                    continue

        filepath = Path(filepath)

        sB = sparse.csr_matrix(B)


        sparse.save_npz(filepath / f'B_matrix_{method_name}_{indicator}_{l.replace("ei_cutoff_3.10_", "").replace("ecoinvent-3.10.1-cutoff", "static").split(" ")[0]}.npz', sB)

        if any(x in l for x in ["2005", "2010", "2020"]):
            sparse.save_npz(filepath / f'B_matrix_{method_name}_{indicator}_{l.replace("ei_cutoff_3.10_", "").replace("NPi", "PkBudg1150").split(" ")[0]}.npz', sB)
            sparse.save_npz(filepath / f'B_matrix_{method_name}_{indicator}_{l.replace("ei_cutoff_3.10_", "").replace("NPi", "PkBudg500").split(" ")[0]}.npz', sB)


In [10]:
fp = r"../windisch/data/lcia"
index = create_index("wind turbines", fp)
create_A_matrix("wind turbines", fp, index)
generate_calculated_values(ei_acts, index, "recipe", "midpoint", fp)
generate_calculated_values(ei_acts, index, "ef", "midpoint", fp)
generate_calculated_values(ei_acts, index, "recipe", "endpoint", fp)

A matrix shape: (74, 74)
Number of methods: 21
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Calculating impacts for 48 activities
B shape: (22, 74)
Number of methods: 25
Calculating impacts for 48 activities
B shape: (25, 74)
Calculating impacts for 48 activities
B shape: (25, 74)
Calculating impacts for 48 activities
B shape: (25, 74)
Calculating impacts for 48 activiti

In [19]:
for c in get_midpoint_methods("recipe"):
    print(c[1])

acidification: terrestrial
climate change
ecotoxicity: freshwater
ecotoxicity: marine
ecotoxicity: terrestrial
energy resources: non-renewable, fossil
eutrophication: freshwater
eutrophication: marine
human toxicity: carcinogenic
human toxicity: non-carcinogenic
ionising radiation
land use
material resources: metals/minerals
ozone depletion
particulate matter formation
photochemical oxidant formation: human health
photochemical oxidant formation: terrestrial ecosystems
water use
energy resources: non-renewable
energy resources: renewable
total
